# Seleção de filtro - Implantação

Este componente realiza a remoção de atributos específicos do dataset.

Este notebook apresenta:
- como carregar modelos e outros resultados do treinamento.
- como utilizar um modelo para fornecer predições em tempo real.

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np
import pandas as pd


class Model(object):
    def __init__(self):
        self.loaded = False
    
    def load(self):
        # Following links explain why to use load() method insted of __init__()

        # Issue associated with sheldon on __init__
        # https://github.com/SeldonIO/seldon-core/issues/2616

        # Solution for the case
        # https://docs.seldon.io/projects/seldon-core/en/latest/python/python_component.html#gunicorn-and-load

        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/filter-selection.joblib")
        self.features = artifacts["columns"]
        self.features_filtered = artifacts["features_filtered"]

    def class_names(self):
        return self.features_filtered.tolist()

    def predict(self, X, feature_names, meta=None):
        # First time load model
        if not self.loaded:
            self.load()

        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)
            X = df[self.features]

        X = X[self.features_filtered]

        return X.to_numpy()